In [21]:
#SCRIPTS PARA ELIMINAR CONTAR Y ELIMINAR SECUENCIAS IGUALES

from collections import Counter
from scipy.stats import mode
import gzip

#Función para realizar la Barrow Wheeler Transform, que genera un tag que nos permitirá buscar secuencias repetidas más fácilmente

def bwt(a):
    words = list(a)
    bwt = []

    for i in range(len(words)):
        word = a[-1] + a[:-1]
        new = ''.join(word)
        a = new
        bwt.append(new)
        i += 1

    sort = sorted(bwt)

    output = []
    for i in range(len(words)):
        element = sort[i]
        last = element[-1]
        i = i + 1
        output.append(last)
    output= "".join(output)
    return output

In [30]:
infile = gzip.open("T2314_KO_24h_S7_R2_001_consensus_over.fastq.gz", "r")
#Inicializar variables
sequence_list = []
sequence_dict = {}
id_list=[] 
qscores_list=[]

#Vamos leyendo el fichero de las secuencias detectadas como repetitivas y creamos una lista con los tags generados por la BWT
#También generamos un diccionario con cada secuencia y su correspondiente tag de BWT para luego poder realizar el paso inverso

while True:
    SequenceID = infile.readline().decode().rstrip("\n")

    if SequenceID == "":
        break
        
    #leemos cada linea y guardamos las variables
    Sequence = infile.readline().decode().rstrip("\n") 
    EmptyLine = infile.readline()
    QualityScores = infile.readline().decode().rstrip("\n")
    
    #Guardamos la BWT de la secuencia y añadimos un registro más al diccionario con secuencia y tag
    result = bwt(Sequence)
    sequence_list.append(result)
    sequence_dict[result] = Sequence
    
    #Guardamos los IDs y los QualityScores de cada secuencia de manera ordenada
    id_list.append(SequenceID)
    qscores_list.append(QualityScores)

infile.close()

In [31]:
#Dado que las secuencias solo tienen el mismo tag si son iguales o son iguales pero con un origen diferente, para eliminar  
#duplicados y contarlos bastará con usar la función Counter sobre la lista de tags
sequence_counts=Counter(sequence_list)

#Generar fastq

In [32]:
result_list = []
import gzip

output_file = gzip.open("T2314_KO_24h_S7_R2_001_filtered.fastq.gz", "wt")

for sequence in sequence_counts:
    #Seleccionamos el SequenceID correspondiente a la primera posición que se halla al buscar cada secuencia en la lista del 
    #contador
    pos=sequence_list.index(sequence)
    id_seq=id_list[pos]
    qscore=qscores_list[pos]
    
    #Obtenemos la cuenta asociada al tag
    count = sequence_counts[sequence]
    
    #Obtenemos la secuencia repetitiva a la que se asocia el tag
    seq = sequence_dict[sequence]
    
    record = "{}\n{}\n+\n{}\n".format(id_seq, seq, qscore)    
    output_file.write(record)
    result_list.append((count, seq, id_seq, qscore ))
output_file.close()
    

In [33]:
#Pasar a Excel
import pandas as pd

# Crear un DataFrame a partir de result_list
df = pd.DataFrame(result_list, columns=['Count', 'Sequence', 'SequenceID', 'QScore'])

# Exportar el DataFrame a un archivo Excel
df.to_excel('T2314_KO_24h_S7_R2_001_filtered.xlsx', index=False)


In [34]:
print(sequence_counts.most_common(10))
print(len(sequence_counts))
print(sum(sequence_counts.values()))
print(len(sequence_counts)/sum(sequence_counts.values()))


[('GAGGCCCCGAAGGAGGCGAGGCGCATTGGCGGTCGGAGGGCGGACCCGGTCCTGCCGGCCCCCCTTGCCTTGCCGCGCCACGCGACCGCCGGGCGCCTGGCCAATGCGTACAAAACGGCGCAGCCCGGAGGG', 5), ('GGTTAGGTGGAGAGAAAGAAGCCGCCCTGCACTGACTCATGACAGTGCCATGCCCCCGGCGCGCGAGGGACGATAGCTGCGAGCGAACCGGAGACCGCGGACATCGCACGCAAATCGTTAGCATGCGATAAA', 4), ('TTGACGCTCGTAATCAGCTTAGGCTTCCGTGTTCTGCTCCCGTGGGCACTAGGAGCCGTTGTTGCCCCCGGCAGCCTGGCGGAGGTCCCGTTGCGTATCTTATTGTCCAGCTTCTCCACCACCGGGACACTG', 3), ('TTGACGCTCGTAATCAGCTTAGGCTTCCGTGTTCTGCTCCCGTGGGCACAGGAGCCGTTGTTGCCCCCGGCAGCCTGGCGGAGGTCCCGTTGCGTATCTTATTGTCTAGCTCTCCACCACCGGGACTATTGC', 3), ('CGAGACCGGGGAAAGGGACGTCAAGAAGCTACACCCTGGGCGTGTGATGCTTTTTTTCGTCGAAGTAACGCTTTGAACAGTGAGAAATAGGAGTCTGGGATTTTAAATAGGAGAGAAATCTCAAAGGAAGTG', 3), ('TCAATGGAAGGAAGCGGGTGGGGACGCGCGCCGCAGGAGGGCGCCTGGGGGGGTGTGGCTTCCCAAGGGGAGTGGAGAGCGGAAGAGAAAGCAAAAAAAACACATACGGAAGAAAAAATAAAGCGCGGGGGG', 2), ('GGAACAGCCGGAAAAGTAAACTAAGTGCGTCAGGCAAATAAAACGACGCCTTAGTGTCACATGTTATGAGAGACTGATGATGTTTTAATGTAAGGATAGGATTCGGGGGCACCTCCTATAGAGGCTCGTGAA', 1), ('GCGATCTCGC

In [35]:
#Buscar coincidencias con el complemento invertido y eliminarlas

from Bio.Seq import Seq

#Hacemos listas auxiliares para luego poder eliminar los duplicados y mantener el orden


secuencias = [elemento[1] for elemento in result_list]
contadores = [elemento[0] for elemento in result_list]
ids = [elemento[2] for elemento in result_list]
qscores = [elemento[3] for elemento in result_list]
secuencias_bwt=list(sequence_counts.keys())

i=0
for secuencia in secuencias:

    secuencia_obj = Seq(secuencia)
    #Hallamos el complemento invertido
    complemento_invertido = secuencia_obj.reverse_complement()
    
    #Realizamos la BWT
    complemento_invertido = bwt(complemento_invertido)
    
    if complemento_invertido in secuencias_bwt:
        coincidencias = [i for i, x in enumerate(secuencias_bwt) if x == complemento_invertido] 
        
        # Eliminar las coincidencias de la lista en orden inverso para no alterar las posiciones de la lista
        for index in reversed(coincidencias):
            #Sumamos los duplicados que vamos a borrar al contador de la secuencia en cuestión
            contadores[i]=contadores[i]+contadores[index]
            
            
            secuencias_bwt.pop(index)            
            secuencias.pop(index)
            contadores.pop(index)
            qscores.pop(index)
            ids.pop(index)
            
    i=i+1      


        

In [36]:
data = {
    'secuencias': secuencias,
    'contadores': contadores,
    'ids': ids,
    'qscores': qscores
}

# Crear el DataFrame a partir del diccionario
df2 = pd.DataFrame(data)

# Guardar el DataFrame en un archivo FASTQ (Con etiqueta "final")
with gzip.open('T2314_KO_24h_S7_R2_001_final.fastq.gz', 'wt') as f:
    for _, row in df2.iterrows():
        f.write(f'{row["ids"]}\n')
        f.write(f'{row["secuencias"]}\n')
        f.write('+\n')
        f.write(f'{row["qscores"]}\n')


In [37]:
#Pasar a Excel
import pandas as pd

# Exportar el DataFrame a un archivo Excel
df2.to_excel('T2314_KO_24h_S7_R2_001_final.xlsx', index=False)
